In [45]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

C:\Users\facun\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\client\session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


https://www.youtube.com/watch?v=1-NYPQw5THU&feature=youtu.be

In [46]:
import pandas as pd
import numpy as np
import datetime
from pandas_summary import DataFrameSummary
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model

In [47]:
df = pd.read_feather('train_normalized_data.fth')
df_test = pd.read_feather('test_normalized_data.fth')

In [48]:
#cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
#            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
#            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'StateHoliday_fw', 'StateHoliday_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']
cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'StateHoliday', 'CompetitionMonthsOpen', 'Promo2Weeks', 
            'StoreType', 'Assortment', 'PromoInterval', 'CompetitionOpenSinceYear', 'Promo2SinceYear', 'State', 
            'Week', 'Events', 'Promo_fw', 'Promo_bw', 'SchoolHoliday_fw', 'SchoolHoliday_bw']
# cat_vars = ['Store', 'DayOfWeek', 'Year', 'Month', 'Day', 'State']


In [49]:
#contin_vars = ['CompetitionDistance', 
#   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
#   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
#   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE',
#   'AfterStateHoliday', 'BeforeStateHoliday', 'Promo', 'SchoolHoliday']

contin_vars = ['CompetitionDistance', 
   'Max_TemperatureC', 'Mean_TemperatureC', 'Min_TemperatureC',
   'Max_Humidity', 'Mean_Humidity', 'Min_Humidity', 'Max_Wind_SpeedKm_h', 
   'Mean_Wind_SpeedKm_h', 'CloudCover', 'trend', 'trend_DE', 'Promo', 'SchoolHoliday']
# contin_vars = []

In [50]:
y_out_columns = ['Sales']

In [51]:
df_train = df[df.Date < datetime.datetime(2015, 7, 1)]  
df_val = df[df.Date >= datetime.datetime(2015, 7, 1)]
print(f'Cantidad en val: {len(df_val)}, porcentaje: {len(df_train)/(len(df_train) + len(df_val))}')
df_train.head

Cantidad en val: 30188, porcentaje: 0.9642465458145908


<bound method NDFrame.head of           index  Store  DayOfWeek       Date  Sales  Customers  Open  \
30188     34565      0          1 2015-06-30   5735        568     1   
30189     34566      1          1 2015-06-30   9863        877     1   
30190     34567      2          1 2015-06-30  13261       1072     1   
30191     34568      3          1 2015-06-30  13106       1488     1   
30192     34569      4          1 2015-06-30   6635        645     1   
...         ...    ...        ...        ...    ...        ...   ...   
844333  1016776    681          1 2013-01-01   3375        566     1   
844334  1016827    732          1 2013-01-01  10765       2377     1   
844335  1016863    768          1 2013-01-01   5035       1248     1   
844336  1017042    947          1 2013-01-01   4491       1039     1   
844337  1017190   1096          1 2013-01-01   5961       1405     1   

           Promo  StateHoliday  SchoolHoliday  ...  AfterStateHoliday_bool  \
30188   1.273237           

In [52]:
X_train = df_train[cat_vars + contin_vars]
X_val = df_val[cat_vars + contin_vars]
X_test = df_test[cat_vars + contin_vars]

In [53]:
X_train.shape
X_val.shape

(30188, 34)

In [54]:
log_output = True
    
if log_output:
    # Escala logaritmica
    max_log_y = np.max(np.log(df[y_out_columns])).values
    y_train = np.log(df_train[y_out_columns].values)/max_log_y
    y_val = np.log(df_val[y_out_columns].values)/max_log_y
else:
    # Normalización
    y_mean = df_train[y_out_columns].mean().values
    y_std = df_train[y_out_columns].std().values
    y_train = (df_train[y_out_columns].values - y_mean)/y_std
    y_val = (df_val[y_out_columns].values - y_mean)/y_std

## Categorical columns transformation

In [55]:
# categoricals_processing = 'no_categoricals'
# categoricals_processing = 'use_onehotencoding'
categoricals_processing = 'use_embeddings'


In [56]:
if categoricals_processing == 'use_embeddings':
    embeddings_model = load_model('embeddings_model.hdf5')
    X_train = embeddings_model.predict(np.hsplit(X_train, X_train.shape[1]), verbose=1)
    X_val = embeddings_model.predict(np.hsplit(X_val, X_val.shape[1]), verbose=1)
    X_test = embeddings_model.predict(np.hsplit(X_test, X_test.shape[1]), verbose=1)

1284/1284 [==============================] - 3s 2ms/step


In [57]:
if categoricals_processing == 'use_onehotencoding':
    # Use One Hot Encoding
    categorical_feature_mask = [col in cat_vars for col in X_train.columns]
    ohe = OneHotEncoder(categorical_features = categorical_feature_mask)
    ohe.fit(X_train)
    X_train = ohe.transform(X_train)
    X_val = ohe.transform(X_val)
    X_test = ohe.transform(X_test)

In [58]:
X_train.shape

(814150, 54)

In [59]:
X_val.shape

(30188, 54)

In [60]:
X_test.shape

(41088, 54)

## XGBoost

In [61]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor

In [62]:
n_estimators=4000
learning_rate=0.25
model = XGBRegressor(n_estimators=n_estimators, learning_rate=learning_rate, 
                     objective='reg:squarederror', n_jobs=12, max_depth=8, gamma=0.005)
model

XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=0.005,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=0.25, max_delta_step=None, max_depth=8,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=4000, n_jobs=12, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
             scale_pos_weight=None, subsample=None, tree_method=None,
             validate_parameters=None, verbosity=None)

In [63]:
fit_params={"early_stopping_rounds":100, 
            "eval_metric" : 'rmse', 
            "eval_set" : [(X_val, y_val.reshape(-1))],
            'verbose': 100,
           }

In [64]:
model.fit(X_train, y_train.reshape(-1), **fit_params)

[0]	validation_0-rmse:0.24679
[100]	validation_0-rmse:0.01825
[181]	validation_0-rmse:0.01829


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0.005, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.25, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=4000, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

# Métrica

$$
\textrm{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}
$$

In [65]:
score = model.score(X_val, y_val)

In [66]:
if log_output:
    y_pred_train = np.exp(model.predict(X_train)*max_log_y)
    y_pred = np.exp(model.predict(X_val)*max_log_y)
    y_pred_test = np.exp(model.predict(X_test)*max_log_y)
else:
    y_pred_train = model.predict(X_train)*y_std + y_mean
    y_pred = model.predict(X_val)*y_std + y_mean
    y_pred_test = model.predict(X_test)*y_std + y_mean

In [67]:
# Train
train_RMSE = np.sqrt((((df_train['Sales'].values - y_pred_train)/df_train['Sales'].values)**2).sum()/len(y_pred_train))

In [68]:
# Validación
val_RMSE = np.sqrt((((df_val['Sales'].values - y_pred)/df_val['Sales'].values)**2).sum()/len(y_pred))

In [69]:
print(score, train_RMSE, val_RMSE)

0.769225321804852 0.26062933102896313 0.20305895705115928


# Baseline

In [70]:
import pandas as pd
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')

In [71]:
stores_mean = {}
for store, g_df in df.groupby('Store'):
    stores_mean[store] = g_df[g_df['Sales'] > 0]['Sales'].mean()

In [72]:
df_test['Sales'] = df_test['Store'].apply(stores_mean.get)
df_test.loc[df_test['Open'] == 0, 'Sales'] = 0

In [73]:
df_test[['Store', 'Sales']].head(10)

,Store,Sales
0,0,4759.096031
1,2,6942.568678
2,6,8817.050891
3,7,5539.358418
4,8,6562.337612
5,9,5568.420918
6,10,8030.977041
7,11,7589.598214
8,12,5034.747182
9,13,5508.567394


In [74]:
df_test[df_test['Open'] == 0][['Store', 'Sales']].head()

,Store,Sales
543,702,0.0
676,878,0.0
840,1096,0.0
1399,702,0.0
1532,878,0.0


In [75]:
sample_csv['Sales'] = df_test['Sales']

In [76]:
sample_csv.to_csv(f'submision_baseline.csv', index=False)

In [77]:
sample_csv.head()

,Id,Sales
0,1,4759.096031
1,2,6942.568678
2,3,8817.050891
3,4,5539.358418
4,5,6562.337612


# Sumbit a la competición

In [78]:
sample_csv = pd.read_csv('dataset/rossmann/sample_submission.csv')
sample_csv['Sales'] = y_pred_test
sample_csv.head()

#sample_csv.to_csv(f'submision_{log_output}-{min_child_samples}-{n_estimators}-{learning_rate}.csv', index=False)
sample_csv.to_csv(f'submision_{log_output}-{n_estimators}-{learning_rate}.csv', index=False)

In [ ]:
cat_vars